In [1]:
import random # random
import pandas as pd # pandas 
import numpy as np # numpy 
import cv2 # opencv
import os 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler


import timm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
import ttach as tta

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore') 


/home/user/anaconda3/envs/test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

CFG = {
    "IMG_SIZE" : 384,
    "EPOCHS" : 100,
    "LEARNING_RATE" : 1e-5,
    "BATCH_SIZE" : 24,
    "SEED" : 42,
}

In [4]:
# fix seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
# dataload
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

In [6]:
# raw_img_path fix 
path = "./train/"
train_df["img_path"] = train_df["img_path"].apply(lambda x : path + x.split("/")[-1])

path = "./up_train/"
train_df["upscale_img_path"] = train_df["upscale_img_path"].apply(lambda x : path + x.split("/")[-1])


path = "./test/"
test_df["img_path"] = test_df["img_path"].apply(lambda x : path + x.split("/")[-1])

path = "./up_test/"
test_df["upscale_img_path"] = test_df["img_path"].apply(lambda x : path + x.split("/")[-1])



In [7]:
# label encoding

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_df["label"])
train_df["label"] = le.transform(train_df["label"])

In [8]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5 , shuffle= True , random_state= CFG["SEED"])
for i, (train_index, test_index) in enumerate(skf.split(train_df["img_path"], train_df["label"])):
     print(f"Fold {i}:")
     print(f"  Train: index={train_index}")
     print(f"  Test:  index={test_index}")
     
     if i == 0:
          break
val_df = train_df.iloc[test_index]
train_df = train_df.iloc[train_index]

Fold 0:
  Train: index=[    0     2     3 ... 15830 15831 15833]
  Test:  index=[    1     4     5 ... 15808 15811 15832]


In [9]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None , flag_mixup = False):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        self.flag_mixup = flag_mixup
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        image = cv2.imread(img_path)
        if self.label_list is not None:
            label = torch.zeros(len(le.classes_))
            label[self.label_list[index]] = 1.
            
        # 기존 image data
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.flag_mixup :
            # mixup에 사용될 data 선택
            mixup_label = torch.zeros(len(le.classes_))

            while True:
              mixup_idx = random.randint(0, len(self.img_path_list)-1) # 전체 데이터 중 아무거나 선택 / 중복되는 클래스가 선택될 수 있음
              if self.label_list[mixup_idx] != self.label_list[index]: # 같은 카테고리 방지
                mixup_label[self.label_list[mixup_idx]] = 1.
                break
        
            # mix할 이미지
            mixup_image = cv2.imread(self.img_path_list[mixup_idx])
            if self.transforms is not None:
                mixup_image = self.transforms(image = mixup_image)["image"]

            # Select a random number from the given beta distribution
            # Mixup the images accordingly
            alpha = 0.4
            lam = np.random.beta(alpha, alpha)
            image = lam * image + (1 - lam) * mixup_image
            label = lam * label + (1 - lam) * mixup_label

        # label one-hot으로 생성
        if self.label_list is not None:
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [10]:
train_transform = A.Compose([
                            A.Resize(CFG["IMG_SIZE"],CFG["IMG_SIZE"],interpolation=2),
                            A.HorizontalFlip(p= 0.5), 
                            A.Rotate(limit =30 , p = 0.5),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG["IMG_SIZE"],CFG["IMG_SIZE"],interpolation=2),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
                            ToTensorV2()
                            ])

In [11]:
train_df["upscale_img_path"] = train_df["upscale_img_path"].apply(lambda x : x.replace("png" , "jpg"))
val_df["upscale_img_path"] = val_df["upscale_img_path"].apply(lambda x : x.replace("png" , "jpg"))

In [12]:

train_dataset = CustomDataset(train_df['img_path'].values, train_df['label'].values, train_transform , flag_mixup = True )
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['img_path'].values, val_df['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [13]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('convnextv2_large.fcmae_ft_in22k_in1k_384', pretrained=True).to(device)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [14]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Macro F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
            print("model save")
            torch.save(best_model.state_dict() , "./models/convnext2_384_64*64_scale_best.pt")
        torch.save(model.state_dict() , "./models/convnext2_384_64*64_scale_last.pt")
    
    return best_model

In [15]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.argmax(1).detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='macro')
        
    
    return _val_loss, _val_score

In [ ]:
model = BaseModel()
model = torch.nn.DataParallel(model).to(device)
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

1. model1 -> convenxtv2

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('convnextv2_large.fcmae_ft_in22k_in1k_384', pretrained=True).to(device)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
CFG["IMG_SIZE"] = 384
test_transform = A.Compose([
                            A.Resize(CFG["IMG_SIZE"],CFG["IMG_SIZE"],interpolation=2),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
load_model = BaseModel().to(device)
load_model = torch.nn.DataParallel(load_model)
load_model.load_state_dict(torch.load('/home/user/바탕화면/train/test/save_code/convnext2_384_64*64_scale_best.pt'))

<All keys matched successfully>

In [ ]:
test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'] * 10, shuffle=False, num_workers=0)

In [ ]:
import numpy as np
from scipy.special import softmax

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            # print(pred)
            preds += softmax(pred.detach().cpu()).tolist()
            del pred , imgs
            # preds += pred.detach().cpu()

    # preds = le.inverse_transform(preds)
    return preds

In [ ]:
tta_ = tta.Compose(
    [
      tta.HorizontalFlip(),
      tta.Multiply(factors=[0.9, 1, 1.1])
    ]
)
tta_model = tta.ClassificationTTAWrapper(load_model, tta_)
preds = inference(tta_model, test_loader, device)

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [07:13<00:00, 14.93s/it]


2. model2 -> eval_336

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('eva_large_patch14_336.in22k_ft_in22k_in1k', pretrained=True).to(device)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
CFG["IMG_SIZE"] = 336
test_transform = A.Compose([
                            A.Resize(CFG["IMG_SIZE"],CFG["IMG_SIZE"],interpolation=2),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
load_model = BaseModel().to(device)
load_model = torch.nn.DataParallel(load_model)
load_model.load_state_dict(torch.load('/home/user/바탕화면/train/test/save_code/eva_336_64*64_scale_best.pt'))

<All keys matched successfully>

In [ ]:
test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'] * 10, shuffle=False, num_workers=0)

In [ ]:
tta_ = tta.Compose(
    [
      tta.HorizontalFlip(),
      tta.Multiply(factors=[0.9, 1, 1.1])
    ]
)
tta_model = tta.ClassificationTTAWrapper(load_model, tta_)
preds_2 = inference(tta_model, test_loader, device)

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [09:14<00:00, 19.12s/it]


3. model3 -> eva02_448

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('eva02_large_patch14_448.mim_in22k_ft_in22k_in1k', pretrained=True).to(device)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
CFG["IMG_SIZE"] = 448
test_transform = A.Compose([
                            A.Resize(CFG["IMG_SIZE"],CFG["IMG_SIZE"],interpolation=2),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
load_model = BaseModel().to(device)
load_model = torch.nn.DataParallel(load_model)
load_model.load_state_dict(torch.load('/home/user/바탕화면/train/test/save_code/eva02_448_64*64_scale_best.pt'))

<All keys matched successfully>

In [ ]:
test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'] * 10, shuffle=False, num_workers=0)

In [ ]:
tta_ = tta.Compose(
    [
      tta.HorizontalFlip(),
      tta.Multiply(factors=[0.9, 1, 1.1])
    ]
)
tta_model = tta.ClassificationTTAWrapper(load_model, tta_)
preds_3 = inference(tta_model, test_loader, device)

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [15:35<00:00, 32.24s/it]


4. model4 -swinv2_384

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('swinv2_large_window12to24_192to384.ms_in22k_ft_in1k', pretrained=True).to(device)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
CFG["IMG_SIZE"] = 384
test_transform = A.Compose([
                            A.Resize(CFG["IMG_SIZE"],CFG["IMG_SIZE"],interpolation=2),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
load_model = BaseModel().to(device)
load_model = torch.nn.DataParallel(load_model)
load_model.load_state_dict(torch.load('/home/user/바탕화면/train/test/save_code/swinv2_384_64*64_scale_best.pt'))

<All keys matched successfully>

In [ ]:
test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'] * 5, shuffle=False, num_workers=0)

In [ ]:
tta_ = tta.Compose(
    [
      tta.HorizontalFlip(),
      tta.Multiply(factors=[0.9, 1, 1.1])
    ]
)
tta_model = tta.ClassificationTTAWrapper(load_model, tta_)
preds_4 = inference(tta_model, test_loader, device)

  0%|          | 0/57 [00:00<?, ?it/s]

100%|██████████| 57/57 [13:38<00:00, 14.35s/it]


5. model5 -> eva_196

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model('eva_large_patch14_196.in22k_ft_in22k_in1k', pretrained=True).to(device)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
CFG["IMG_SIZE"] = 196
test_transform = A.Compose([
                            A.Resize(CFG["IMG_SIZE"],CFG["IMG_SIZE"],interpolation=2),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
load_model = BaseModel().to(device)
load_model = torch.nn.DataParallel(load_model)
load_model.load_state_dict(torch.load('/home/user/바탕화면/train/test/extra_code/eva_196_64*64_scale_best.pt'))

<All keys matched successfully>

In [ ]:
test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'] * 10, shuffle=False, num_workers=0)

In [ ]:
tta_ = tta.Compose(
    [
      tta.HorizontalFlip(),
      tta.Multiply(factors=[0.9, 1, 1.1])
    ]
)
tta_model = tta.ClassificationTTAWrapper(load_model, tta_)
preds_5 = inference(tta_model, test_loader, device)

  0%|          | 0/29 [00:00<?, ?it/s]

100%|██████████| 29/29 [05:59<00:00, 12.40s/it]


# 앙상블 

In [ ]:
result = (np.array(preds) + np.array(preds_2) + np.array(preds_3)
          + np.array(preds_4) + np.array(preds_5)).argmax(axis=1)

In [ ]:
result = le.inverse_transform(result)

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['label'] = result
submission.to_csv('./0506-5models_ensemble3.csv',index=False)